In [ ]:
import torch
from PIL import Image

from colpali_engine.models import ColPali, ColPaliProcessor
from transformers import BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

model_name = "vidore/colpali-v1.2"

# help(ColPali.from_pretrained)
# help(ColPaliProcessor.from_pretrained)

model = ColPali.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="cuda:0",
    quantization_config=quantization_config,
).eval()

processor = ColPaliProcessor.from_pretrained(model_name, quantization_config=quantization_config)


In [ ]:
# These both have to be arrays for processing :( 
images = [
    Image.open("./screenshots/mob.jpg")
]

queries = [
    "mobile"
]

# Process the inputs
batch_images = processor.process_images(images).to(model.device)
batch_queries = processor.process_queries(queries).to(model.device)

print(type(processor))


In [ ]:
# Forward pass
# torch.cuda.empty_cache()
with torch.no_grad():
    image_embeddings = model(**batch_images)
    query_embeddings = model(**batch_queries)


np_image_embeddings = image_embeddings.cpu().detach().numpy() #img emb in numpy
np_query_embeddings = query_embeddings.cpu().detach().numpy() #query emb in numpy

print(np_image_embeddings)

scores = processor.score_multi_vector(query_embeddings, image_embeddings)
scores


In [1]:
from vectors import get_img_emb, get_text_emb

# print(get_img_emb("./screenshots/mob.jpg"))
# print(get_text_emb("mobile"))

img_emb = get_img_emb("./screenshots/mob.jpg")
text_emb = get_text_emb("iphone")

print("Image emb: ", img_emb.shape)
print("Text emb: ", text_emb.shape)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Image emb:  (1, 1030, 128)
Text emb:  (1, 15, 128)
